In [16]:
import cbstools as cbs
from os.path import join
import os
import nibabel as nb
import numpy as np
import glob as glob
import pandas as pd

In [6]:
cbs.filter_sigmoid(np.array([1000000,.002,.0015]))

array([ 1.        ,  0.5       ,  0.26894142])

In [35]:
## MD SIGMOID FUNCTION PARAMS:
#sigmoid must scale from 0 to 1, hopefully
# center x at 0.002 and slope at 0.0005

# y(+infin)=1, (-infin)=0, .002=.5, .0015=1/4
# p=((x-.002)/.0005)
# y=1/(1+exp(-p))

current_dir = os.getcwd() #current directy of this notebook

data_dir='/home/chris/Documents/code/python/cbstools-python/test-python'
out_dir='/home/chris/Documents/code/python/cbstools-python/test-python/out'

t1_fname='t1map_stripped.nii.gz'
uni_fname='uni_stripped.nii.gz'
pre_fname='filters.nii.gz'

atlas_file = 'brain-atlas-3.0.3.txt'

print(cbs.ATLAS_DIR) #as defined in your defaults file
print(cbs.TOPOLOGY_LUT_DIR)

/home/chris/mipav/plugins/atlases/brain-segmentation-prior3.0/
/home/chris/Documents/code/python/cbstools-python/ToloplogyLUT/


In [36]:
#cbs=reload(cbs)
cbs.setup_JVM()

Java virtual machine successfully started.


In [372]:
cbs=reload(cbs)
cbs.MGDMBrainSegmentation([[join(data_dir,uni_fname),"MP2RAGE7T"]],
                             output_dir=out_dir,atlas_file=None, topology_lut_dir=None)

Thank you for choosing the MGDM segmentation from the cbstools for your brain segmentation needs
Sit back and relax, let the magic of algorithms happen...

Atlas file: /home/chris/mipav/plugins/atlases/brain-segmentation-prior3.0/brain-atlas-3.0.3.txt
Topology LUT durectory: /home/chris/Documents/code/python/cbstools-python/ToloplogyLUT/

Input files and filetypes:
  1  ['/home/chris/Documents/code/python/cbstools-python/test-python/uni_stripped.nii.gz', 'MP2RAGE7T']
data orientation: ('L', 'P', 'S') slice settings: AXIAL
mgdm orientation: [[ 0.  1.]
 [ 1.  1.]
 [ 2.  1.]]
data orientation: [[ 0. -1.]
 [ 1. -1.]
 [ 2.  1.]]
Executing MGDM on your inputs
Don't worry, the magic is happening!
/home/chris/Documents/code/python/cbstools-python/test-python/out/uni_stripped_seg_cjs.nii.gz
Data stored in: /home/chris/Documents/code/python/cbstools-python/test-python/out
Execution completed


(array([[[1, 1, 1, ..., 1, 1, 1],
         [1, 1, 1, ..., 1, 1, 1],
         [1, 1, 1, ..., 1, 1, 1],
         ..., 
         [1, 1, 1, ..., 1, 1, 1],
         [1, 1, 1, ..., 1, 1, 1],
         [1, 1, 1, ..., 1, 1, 1]],
 
        [[1, 1, 1, ..., 1, 1, 1],
         [1, 1, 1, ..., 1, 1, 1],
         [1, 1, 1, ..., 1, 1, 1],
         ..., 
         [1, 1, 1, ..., 1, 1, 1],
         [1, 1, 1, ..., 1, 1, 1],
         [1, 1, 1, ..., 1, 1, 1]],
 
        [[1, 1, 1, ..., 1, 1, 1],
         [1, 1, 1, ..., 1, 1, 1],
         [1, 1, 1, ..., 1, 1, 1],
         ..., 
         [1, 1, 1, ..., 1, 1, 1],
         [1, 1, 1, ..., 1, 1, 1],
         [1, 1, 1, ..., 1, 1, 1]],
 
        ..., 
        [[1, 1, 1, ..., 1, 1, 1],
         [1, 1, 1, ..., 1, 1, 1],
         [1, 1, 1, ..., 1, 1, 1],
         ..., 
         [1, 1, 1, ..., 1, 1, 1],
         [1, 1, 1, ..., 1, 1, 1],
         [1, 1, 1, ..., 1, 1, 1]],
 
        [[1, 1, 1, ..., 1, 1, 1],
         [1, 1, 1, ..., 1, 1, 1],
         [1, 1, 1, ..., 1, 1, 

In [15]:
cbs=reload(cbs)
cbs.MGDMBrainSegmentation([[join(data_dir,uni_fname),"MP2RAGE7T"],[join(data_dir,pre_fname),"Filters"]],
                             output_dir=out_dir,atlas_file=None, topology_lut_dir=None)


Thank you for choosing the MGDM segmentation from the cbstools for your brain segmentation needs
Sit back and relax, let the magic of algorithms happen...

Atlas file: /home/chris/mipav/plugins/atlases/brain-segmentation-prior3.0/brain-atlas-3.0.3.txt
Topology LUT durectory: /home/chris/Documents/code/python/cbstools-python/ToloplogyLUT/

Input files and filetypes:
  1  ['/home/chris/Documents/code/python/cbstools-python/test-python/uni_stripped.nii.gz', 'MP2RAGE7T']
Input files and filetypes:
  2  ['/home/chris/Documents/code/python/cbstools-python/test-python/filters.nii.gz', 'Filters']
Executing MGDM on your inputs
Don't worry, the magic is happening!
/home/chris/Documents/code/python/cbstools-python/test-python/out/uni_stripped_seg_cjs.nii.gz
Data stored in: /home/chris/Documents/code/python/cbstools-python/test-python/out
Execution completed


In [3]:
def seg_erode(seg_d, iterations=1, background_idx = 1,
              structure=None, min_vox_count = 5,seg_null_value = 0,
             VERBOSE=False):
    # erode indices (integers) to identify "core" structure
    # XXX might need to limit erosion here and loop myself
    # default erosion structure is 3,1 (which is not super restrictive, and should work for most)
    # seg null value is int value that is assigned to voxels that were eroded from the segmentation
    import scipy.ndimage as ndi
    import numpy as np
    
    if structure is None:
        structure = ndi.morphology.generate_binary_structure(3, 1)
    if seg_null_value == 0:
        seg_shrunk_d = np.zeros_like(seg_d)
        temp_d = np.zeros_like(seg_d)
    else:
        seg_shrunk_d = np.ones_like(seg_d)*seg_null_value
        temp_d = np.ones_like(seg_d)*seg_null_value    
    
    seg_idxs = np.unique(seg_d)
    
    if seg_null_value in seg_idxs:
        print("Shit, your null value is also an index. This will not work.")
        print("Set it to a suitably strange value that is not already an index. {0,999}")
        return None
    
    for seg_idx in seg_idxs:
        print(seg_idx),
        if (background_idx is not None) and (background_idx == seg_idx):
            seg_shrunk_d[seg_d==seg_idx] = seg_idx #just set the value to the bckgrnd value, and be done with it
            if VERBOSE:
                print("[bckg]"),
        else:
            temp_d[seg_d==seg_idx] = 1
            for idx in range(0, iterations): #messy, does not exit the loop when already gone too far.
                temp_temp_d = ndi.binary_erosion(temp_d, iterations=1, structure=structure)
                if np.sum(temp_temp_d) >= min_vox_count:
                    temp_d = temp_temp_d
                    if VERBOSE:
                        print("[y]"),
                else:
                    if VERBOSE:
                        print("[no]"),
            seg_shrunk_d[temp_d==1] = seg_idx
            temp_d[:,:,:] = seg_null_value
            if VERBOSE:
                print(seg_idx)
    print("")
    return seg_shrunk_d



def extract_metrics_from_seg(seg_d, metric_d, norm_data = True,
                             background_idx = 1, seg_null_value = 0,
                             percentile_top_bot = [75,25],
                             return_normed_metric_d=False):
    #returns np matrix of indices, and one of median, and percentiles
    #norm_data = true first zscores all of the data other than the background
    import numpy as np
    import scipy
    seg_idxs = np.unique(seg_d)
    res = np.zeros((len(seg_idxs),3))
    
    if norm_data: # rescale the data to 0
        if background_idx is not None: #we need to exclude the background data from the norming
            metric_d[seg_d!=background_idx] = (metric_d[seg_d!=background_idx] - np.min(metric_d[seg_d!=background_idx])) / (np.max(metric_d[seg_d!=background_idx]) - np.min(metric_d[seg_d!=background_idx]))
        else:
            metric_d = (metric_d - np.min(metric_d)) / (np.max(metric_d)-np.min(metric_d))
        
    for idx,seg_idx in enumerate(seg_idxs):
        if (background_idx is not None) and ((seg_idx == background_idx) or (seg_idx == seg_null_value)):
            res[idx,:] = [0,0,0]
        else:
            d_1d = np.ndarray.flatten(metric_d[seg_d == seg_idx])
            res[idx,:] = [np.mean(d_1d),
                          np.percentile(d_1d,np.max(percentile_top_bot)),
                          np.percentile(d_1d,np.min(percentile_top_bot))]
    if return_normed_metric_d:
        return seg_idxs,res,metric_d
    else:
        return seg_idxs,res

In [8]:
# load the output segmentatinb.Nifti1Image(d_seg_ero,a_seg).to_filename(os.path.join(out_dir,"d_seg_ero.nii.gz"))on file
seg_files = glob.glob(os.path.join(out_dir,"*_seg_cjs.nii.gz"))
# point it back at the input file, extract values from 
seg_file=seg_files[0]
img=nb.load(seg_file)
d_seg = img.get_data()
a_seg = img.affine

d_metric = nb.load(join(data_dir,uni_fname)).get_data()
seg_null_value = 0

print("seg indices: "),
np.unique(d_seg)
np.where(d_seg==50)
#d_seg[70,66,43]

seg indices: 

(array([ 14,  14,  14, ..., 132, 132, 132]),
 array([ 86,  87,  87, ..., 103, 103, 103]),
 array([60, 51, 58, ..., 57, 58, 59]))

In [15]:
img.header.get_data_shape()

(146, 174, 146)

In [5]:
#erode the seg file
cbs=reload(cbs)
d_seg_ero = cbs.seg_erode(d_seg,background_idx = 1, seg_null_value = seg_null_value)
nb.Nifti1Image(d_seg_ero,a_seg).to_filename(os.path.join(out_dir,"d_seg_ero.nii.gz"))

#extract the values from each index
[seg_idxs,seg_stats,d_metric_norm] = cbs.extract_metrics_from_seg(d_seg_ero,d_metric,seg_null_value=seg_null_value,
                                                             return_normed_metric_d=True)
nb.Nifti1Image(d_metric_norm,a_seg).to_filename(os.path.join(out_dir,"d_metric_norm.nii.gz"))

1 8 10 11 12 13 14 17 18 20 25 26 27 30 31 32 33 34 35 36 37 38 39 40 41 43 46 47 48 50


In [6]:
## explicitly set our median and spread variables for priors, just makes it easier to put into df later

MIN_QUART_DIFF = 0.10
prior_medians=seg_stats[:,0]
prior_quart_diffs=np.squeeze(np.abs(np.diff(seg_stats[:,1:3])))

# fill in prior_quart_diffs that are below a certain value??
prior_quart_diffs[prior_quart_diffs<MIN_QUART_DIFF] = MIN_QUART_DIFF
print(seg_idxs)
print(prior_medians)
print(prior_quart_diffs)



[ 0  1  8 10 11 12 13 14 17 18 20 25 26 27 30 31 32 33 34 35 36 37 38 39 40
 41 43 46 47 48 50]
[ 0.39642414  2.48996043  0.          0.22563398  0.08410583  0.08987724
  0.16868669  0.20622344  0.48463598  0.17966332  0.2674751   0.46390796
  0.45047241  0.46238199  0.39889997  0.34720081  0.48333097  0.46535555
  0.35564139  0.36721775  0.56746542  0.54524189  0.68753362  0.65726626
  0.78753018  0.7920326   0.69178802  0.75529468  0.8249054   0.83420277
  0.06309689]
[ 0.48501778  0.1         0.1         0.24719246  0.1         0.1
  0.18838063  0.18998904  0.1         0.1         0.30075012  0.21800042
  0.32179964  0.29512179  0.17577323  0.18397549  0.1         0.10100538
  0.15619811  0.15573829  0.13367768  0.13912316  0.12773302  0.1336633
  0.1         0.1         0.11633818  0.1         0.1         0.1         0.1       ]


In [11]:
## OLD WAY
## parse the atlas file to get the lut and the intensity priors of interest

#identify the start and stop locations for the LUT and the intensity priors of interest
fp = open(os.path.join(cbs.ATLAS_DIR,atlas_file))
for i, line in enumerate(fp):
    if "Structures:" in line: #this is the beginning of the LUT
        lut_idx = i
        lut_rows=map(int,[line.split()[1]])[0]
#    if "Topology Atlas:" in line: #the end of the LUT #OR you could use the value in the line?
#        lut_idx[1] = i-2
    if "Intensity Prior:" in line:
        if contrast_name in line:
            con_idx = i
fp.close()

lut=pd.read_csv(os.path.join(cbs.ATLAS_DIR,atlas_file),sep="\t+",
                skiprows=lut_idx+1,nrows=lut_rows,engine='python',
                names=["Index","Type"])

#con_idx[1] = len(lut) #total number is the same length as the lut
priors=pd.read_csv(os.path.join(cbs.ATLAS_DIR,atlas_file),sep="\t+",
                   skiprows=con_idx+1,nrows=lut_rows,engine='python',
                   names=["Median","Spread","Weight"])


In [15]:
cbs=reload(cbs)
new_atlas_file = 'brain-atlas-3.0.3_cjs.txt'
contrast_name = 'Mprage3T'
[con_idx,lut_rows,lut,priors]=cbs.extract_lut_priors_from_atlas(new_atlas_file,contrast_name)

In [20]:
## create a new dataframe and populate with our new values
# could also add intelligence here, to only push it to 1/2 of the difference etc (next release...)
priors_new = pd.DataFrame.copy(priors)
for idx in lut.Index:
    priors_new[lut["Index"]==idx] = [prior_medians[seg_idxs==idx], prior_quart_diffs[seg_idxs==idx],1]
priors_new.head()
priors_new_string = priors_new.to_csv(sep="\t",header=False,float_format="%.2f")
priors_new_string_lines = priors_new_string.split("\n")[0:-1] #convert to list of lines, cut the last empty '' line


In [36]:
#keep track of what we did with the old and new priors
#assume that everything is in the correct order... if it is not, things are not so good!
#this allows looping across individuals or successive segmentations, but not currently both

all_Ss_priors_list_median=np.vstack((priors.Median,priors_new.Median))
all_Ss_priors_list_spread=np.vstack((priors.Spread,priors_new.Spread))

np.diff(all_Ss_priors_list_median,axis=0)

array([[ 0.        , -0.43690311, -0.95      ,  0.04563398, -0.14589417,
        -0.14012276, -0.06131331, -0.02377656, -0.01536402, -0.32033668,
        -0.2325249 , -0.12609204, -0.07952759, -0.06761801, -0.29110003,
        -0.34279919, -0.20666903, -0.22464445, -0.27435861, -0.26278225,
        -0.21253458, -0.23475811, -0.09246638, -0.12273374, -0.16246982,
        -0.1579674 , -0.23821198, -0.16470532, -0.1050946 , -0.09579723]])

In [9]:
# write out the newly altered priors list
fp = open(os.path.join(cbs.ATLAS_DIR,atlas_file))
fp_new = open(os.path.join(cbs.ATLAS_DIR,new_atlas_file),"w")
ii=0
#only replace the lines that we changed
for i, line in enumerate(fp):
    if i > con_idx[0] and i<con_idx[1]+con_idx[0]:
        fp_new.write(priors_new_string_lines[ii]+"\n")
        ii+=1
    else:
        fp_new.write(line)
fp.close()
fp_new.close()
print("New atlas file written to: " + fp_new.name)

New atlas file written to: /home/chris/mipav/plugins/atlases/brain-segmentation-prior3.0/brain-atlas-3.0.3_cjs.txt


In [54]:
## more testing, with FA and MD files
cbs=reload(cbs)
in_dir='/home/chris/Documents/projects_working/2016_Brainhack/DWI_sample'
from os.path import join

md_filt=cbs.filter_sigmoid(join(in_dir,'CIC_dti_MD.nii.gz'))
cbs.niiSave(join(in_dir,'CIC_dti_MD_sig.nii.gz'),md_filt[0],affine=md_filt[1],header=md_filt[2])

'/home/chris/Documents/projects_working/2016_Brainhack/DWI_sample/CIC_dti_MD_sig.nii.gz'

In [369]:
cbs=reload(cbs)
con1_file = join(in_dir,'CIC_dti_FA.nii.gz')
con2_file = join(in_dir,'CIC_dti_MD_sig.nii.gz')
seg,aff,head = cbs.MGDMBrainSegmentation([[con1_file,'DWIFA3T'],[con2_file,'DWIMD3T']],
                                  output_dir=in_dir,atlas_file=None, 
                                  topology_lut_dir=None,num_steps=0)


Thank you for choosing the MGDM segmentation from the cbstools for your brain segmentation needs
Sit back and relax, let the magic of algorithms happen...

Atlas file: /home/chris/mipav/plugins/atlases/brain-segmentation-prior3.0/brain-atlas-3.0.3.txt
Topology LUT durectory: /home/chris/Documents/code/python/cbstools-python/ToloplogyLUT/

Input files and filetypes:
  1  ['/home/chris/Documents/projects_working/2016_Brainhack/DWI_sample/CIC_dti_FA.nii.gz', 'DWIFA3T']
orientation: ('L', 'A', 'S') slice settings: AXIAL
Input files and filetypes:
  2  ['/home/chris/Documents/projects_working/2016_Brainhack/DWI_sample/CIC_dti_MD_sig.nii.gz', 'DWIMD3T']
orientation: ('L', 'A', 'S') slice settings: AXIAL
Executing MGDM on your inputs
Don't worry, the magic is happening!
/home/chris/Documents/projects_working/2016_Brainhack/DWI_sample/CIC_dti_FA_seg_cjs.nii.gz
Data stored in: /home/chris/Documents/projects_working/2016_Brainhack/DWI_sample
Execution completed


In [368]:
cbs=reload(cbs)
#sform = nb.load(join(in_dir,con1_file)).get_sform()
#sform[2,2] = sform[2,2]*-1
seg_flip, aff_flip = cbs.flip_affine_data_orientation(seg,aff,flipLR=False,flipAP=True,flipIS=False)
seg_flip2 = nb.orientations.apply_orientation(seg,aff_flip)
img=nb.Nifti1Image(nb.orientations.apply_orientation(seg,aff_flip),aff_flip)
#img.set_sform(sform)
img.to_filename(join(in_dir,"test2.nii.gz"))

OrientationError: Data array has fewer dimensions than orientation

In [220]:
ornt = nb.orientations.io_orientation(np.diag([-1, 1, 1, 1]).dot(aff)) #LAS
print(ornt)
print(aff)
#t_aff = nb.orientations.inv_ornt_aff(ornt,np.shape(seg))

[[ 0.  1.]
 [ 1.  1.]
 [ 2. -1.]]
[[ -1.99554968e+00   9.41075459e-02   9.44717824e-02   1.20489822e+02]
 [  9.39716771e-02   1.99778461e+00  -5.09611191e-03  -1.08620071e+02]
 [  9.46069360e-02   6.45936525e-04  -1.99776065e+00  -4.16052132e+01]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]


In [345]:
from nibabel.orientations import io_orientation, inv_ornt_aff, apply_orientation, ornt_transform
#mgdm_orient = nb.orientations.io_orientation(np.diag([1, 1, 1, 1]))
ornt_input = io_orientation(aff)
ornt_input2mgdm = nb.orientations.io_orientation(np.diag([-1, -1, 1, 1]).dot(aff)) #1 1 1 is RAS, -1 -1 1 LPS (mgdm default)
print(ornt_input2mgdm)
#ornt_input2mgdm[:,1]=ornt_input2mgdm[:,1]*ornt_input[:,1]
if np.all(ornt == [[0, 1],
                   [1, 1],
                   [2, 1]]):  # already in LAS+
    t_aff = np.eye(4)
    print("All good")
else: 
    #ornt_input2mgdm[:,1]=ornt_input2mgdm[:,1]*-1
    t_aff = inv_ornt_aff(ornt_input2mgdm, np.shape(seg))
    print("no good")
print(aff)
print(ornt_input2mgdm)
print(t_aff)
aff2[1,-1]=aff2[1,-1]+48
aff_new = np.dot(aff, t_aff)
print(aff_new)
aff2=aff

seg_flip = apply_orientation(seg, ornt_input2mgdm)
nb.Nifti1Image(seg_flip,aff2).to_filename(join(in_dir,"xxx_test2_flip.nii.gz"))

[[ 0.  1.]
 [ 1. -1.]
 [ 2.  1.]]
no good
[[ -1.99554968e+00   9.41075459e-02   9.44717824e-02   1.20489822e+02]
 [  9.39716771e-02   1.99778461e+00  -5.09611191e-03   0.00000000e+00]
 [  9.46069360e-02   6.45936525e-04   1.99776065e+00  -4.16052132e+01]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]
[[ 0.  1.]
 [ 1. -1.]
 [ 2.  1.]]
[[   1.    0.    0.    0.]
 [   0.   -1.    0.  127.]
 [   0.    0.    1.    0.]
 [   0.    0.    0.    1.]]
[[ -1.99554968e+00  -9.41075459e-02   9.44717824e-02   1.32441481e+02]
 [  9.39716771e-02  -1.99778461e+00  -5.09611191e-03   3.01718646e+02]
 [  9.46069360e-02  -6.45936525e-04   1.99776065e+00  -4.15231792e+01]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]


In [367]:
#current try, with my stuffs...
from nibabel.orientations import io_orientation, inv_ornt_aff, apply_orientation, ornt_transform

ornt_orig = io_orientation(aff)
ornt_mgdm = io_orientation(np.diag([-1, -1, 1, 1]).dot(aff)) #-1 -1 1 LPS (mgdm default); 1 1 1 is RAS
ornt_chng = ornt_transform(ornt_mgdm,ornt_orig) #to get from MGDM to our original input
#ornt_chng[:,1]=ornt_chng[:,1]*-1
#t_aff = np.diag(np.append(ornt_chng[:,1],[1]))
#aff_new = np.dot(aff, t_aff)
print(ornt_orig)
print(ornt_chng)
print(t_aff)
aff2 = np.copy(aff)
aff2[1,-1]=aff2[1,-1]-41
seg_flip = apply_orientation(seg, ornt_chng)
img=nb.Nifti1Image(seg_flip,aff)
img.update_header()
img.to_filename(join(in_dir,"xxx_test2_flip.nii.gz"))
print(aff)
print(aff2)

[[ 0. -1.]
 [ 1.  1.]
 [ 2.  1.]]
[[ 0. -1.]
 [ 1. -1.]
 [ 2.  1.]]
[[   1.    0.    0.    0.]
 [   0.   -1.    0.  127.]
 [   0.    0.    1.    0.]
 [   0.    0.    0.    1.]]
[[ -1.99554968e+00   9.41075459e-02   9.44717824e-02   1.20489822e+02]
 [  9.39716771e-02   1.99778461e+00  -5.09611191e-03  -4.80000000e+01]
 [  9.46069360e-02   6.45936525e-04   1.99776065e+00  -4.16052132e+01]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]
[[ -1.99554968e+00   9.41075459e-02   9.44717824e-02   1.20489822e+02]
 [  9.39716771e-02   1.99778461e+00  -5.09611191e-03  -8.90000000e+01]
 [  9.46069360e-02   6.45936525e-04   1.99776065e+00  -4.16052132e+01]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]


In [356]:
flirt_aff = [[0.9999988322,  -0.0008334219106,  0.001260845295,  0.02879772489],  
[0.001064651508 , 0.9805542622,  -0.1962452202,  -3.774127818],  
[-0.001072772206 , 0.1962463325,  0.9805540593,  -27.28980765],  
[0,  0,  0,  1  ]]

In [365]:
np.shape(seg_flip)

(128, 128, 62)

In [311]:
np.append(a,[1])

array([-1., -1.,  1.,  1.])

In [205]:
img3=nb.load('/home/chris/Documents/projects_working/2016_Brainhack/DWI_sample/CIC_dti_FA_seg_cjs.nii.gz')
print(img3.header)

<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : 32
db_name         : 
extents         : 0
session_error   : 0
regular         : r
dim_info        : 0
dim             : [  3 128 128  62   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : uint32
bitpix          : 32
slice_start     : 0
pixdim          : [-1.          2.          2.          1.99999976  1.          0.          0.
  0.        ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 10
cal_max         : 0.00220409687608
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : FSL5.0
aux_file        : 
qform_code      : scanner
sform_code      : scanner
quatern_b       : -0.0235229991376
quatern_c       : -0.999443292618
quatern_d       : -0.000718155759387
qoffset_x

In [178]:
#FAseg[1][2,2]=FAseg[1][2,2]*-1
#img=nb.Nifti1Image(seg,cbs.flip_affine_orientation(FAseg[1],flipLR=False,flipAP=True)
#img.set_data_dtype('uint32')
#newq=img.get_qform()[2,2]*img.get_qform
#img.set_qform(newq)
#img.to_filename(join(in_dir,"test.nii.gz"))

In [176]:
newq=img.get_qform()
newq[2,2]=img.get_qform()[2,2]*-1

In [200]:
nb.apply_orientation?

In [142]:
cbs.get_affine_orientation_slice(nb.load(join(data_dir,uni_fname)).affine)

(('L', 'P', 'S'), 'AXIAL')

In [158]:
#cbs.recursively_generate_group_intensity_priors(seg_iterations=0)
na=nb.load(join(data_dir,uni_fname)).header
na['datatype'] = np.array(32).astype('uint32')
print(na)

<class 'nibabel.nifti1.Nifti1Header'> object, endian='>'
sizeof_hdr      : 348
data_type       : 
db_name         : uni_stripped.nii
extents         : 0
session_error   : 0
regular         : r
dim_info        : 0
dim             : [  3 146 174 146   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : complex64
bitpix          : 32
slice_start     : 0
pixdim          : [ 1.          1.24657536  1.25114942  1.24657536  0.          0.          0.
  0.        ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 10
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 4096
glmin           : 0
descrip         : Unknown Modality
aux_file        :  
qform_code      : scanner
sform_code      : unknown
quatern_b       : 0.0
quatern_c       : 0.0
quatern_d       : 1.0
qoffset_x       : 107.212188721